# Code Portfolio Summative Assignment

## Introduction

Over the last 10 weeks, you have built a simulation of a mobile robot which you will submit for assessment. Week-by-week, you have added the following elements to your simulation:

- Week 1: Storing information about a robot (e.g. robot name and size), calculating turning angle of the robot
- Week 2 (P1): Robot motion: calculating new position (x,y) and orientation/heading ($\theta$) from left and right wheel speed
- Week 3: Generating the 'lawnmower' search pattern 
- Week 4 (P2):  Making a library: *refactoring* your code
- Week 5 (C1): Autonomous Robot Navigation: Goal seeking and obstacle avoidance (extension task: bug algorithms)
- Week 6: Reading week
- Week 7: Avoiding multiple obstacles
- Week 8 (P3): Robot log file 
- Week 9: C2
- **Week 10 (P4): Sensing**

In this assessment, you will add a final element, sensing to your simulation. 

**The mark for your assessment will be based entirely on the code you write for P4.** 

**We have given you a file, *ExerciseP4.py*, which contains the sample answers for P1, P2 and P3.** 

**As this is a summative assignment, we strongly recommend that you start from our sample answers, rather than your own code for P1-3.**

You will also need the latest version of robot_plotter.py in the same directory as your program. 



## Background

Until now, we have simulated a robot with *omnidirectional* sensing. In other words, the robot has been able to detect any obstacle that comes within a specified radius of the robot centre. 

<img src="https://github.com/engmaths/SEMT10002_2025/blob/main/media/week_10/omnidirectional_sensing.png?raw=true" width="50%">

In reality, robots need to use sensors to gather information about their environment. There are many different types of sensors that could be used to detect obstacles, but many distance sensors will:

1. Point in a single direction relative to the robot heading.
2. Have a range (beyond which objects can't be detected).

This means that rather than being able to sense obstacles in *any* direction, many robots can only see obstacles if they are in front / to the side of the robot. 

Today you are going to update the simulation to model a robot with three obstacle detection sensors. 

The sensors point in directions $+\frac{\pi}{4}$ radians, $0$ radians, and $-\frac{\pi}{4}$ radians respectively, *relative to* the robot heading, $\theta$. 

The sensors each have a range of 300mm, measured from the robot centre. 

<img src="https://github.com/engmaths/SEMT10002_2025/blob/main/media/week_10/robot_sensors.png?raw=true" width="50%">


## Task

Modify the `detect_obstacles` function so that an obstacle or wall is only detected if it's *edge* intersects the line representing the detection range of one or more of the three sensors.

Remeber to check each edge of the wall/obstacle

Your modified function header should have the following form. Notice that compared to the example solution in *ExerciseP4.py*:
- there are two new input arguments `sensor_angles`, `sensor_range`
- the function outputs are unchanged, it returns `obstacle_detected`, `obstacle_type`

```python
def detect_obstacles(   robot_x_position, 
                        robot_y_position, 
                        sensor_angles,
                        sensor_range,
                        obstacles,
                        map_x_min=map_x_min,
                        map_x_max=map_x_max,
                        map_y_min=map_y_min,
                        map_y_max=map_y_max):
    
    """
    Determines whether the robot has collided with a wall or obstacle by checking 
    for intersections between the sensor’s detection line and the line segments 
    that represent walls or obstacles.

    Parameters
    ----------
    robot_x_position (int or float): x coordinate of robot
    robot_y_position (int or float): y coordinate of robot
    sensor_angles(list of ints or floats): The angle of each sensor relative to the robot heading 
    sensor_range(int or float): The range within which obstacles can be detected by each sensor
    obstacles (list of tuples): List of obstacle positions and dimensions.
                                Each obstacle is represented as ((position_x, position_y)
    map_x_min (int or float) : Minimum x coordinate of rectangular boundary
    map_x_max (int or float) : Maximum x coordinate of rectangular boundary
    map_y_min (int or float) : Minimum y coordinate of rectangular boundary
    map_y_max (int or float) : Maximum y coordinate of rectangular boundary

    Returns
    -------
    obstacle_detected (Boolean): True if obstacle detected, else False
    obstacle_type (str): The type of obstacle detected 'obstacle' or 'wall'
    """
    ```

### An algorithm to test if two lines intersect 

Collision with an obstacle can be represented as an *intersection* between the line representing the sensor and the edge of an obstacle or boundary. These two lines give us four points, which can be connected by two sets of matching triangles (i.e. 4 possible triangles in total). If the orientations for the matching sets are different, then the lines must overlap.



#### Orientation

The orientation function is used to determine the relative orientation of three ordered points (A,B,C) in a 2D plane: clockwise, counterclockwise, or collinear. 

<img src="https://github.com/engmaths/SEMT10002_2025/blob/main/media/week_10/cw_ccw_colinear.png?raw=true" width="50%">


<!-- The ordered points can be represented as two vectors<br>
$\vec{AB} = (x_B - x_A, y_B - y_A)$<br>
$\vec{AC} = (x_C - x_A, y_C - y_A)$ -->

<!-- $$
\begin{aligned}
 {\rm orientation}(A, B, C) &= \vec{AB} \times \vec{AC} \\
  &= (x_B - x_A, y_B - y_A) \times  (x_C - x_A, y_C - y_A) \\
  &= (x_B - x_A)(y_C - y_A) - (y_B - y_A)(x_C - x_A)
\end{aligned}
$$ -->

If the coordinates of points $A$, $B$ and $C$ are $(x_A, y_A)$, $(x_B, y_B)$ and $(x_C, y_C)$, then:

$$
\begin{aligned}
 {\rm orientation}(A, B, C) &= (x_B - x_A)(y_C - y_A) - (y_B - y_A)(x_C - x_A)
\end{aligned}
$$

If ${\rm orientation}(A, B, C) = 0$, the points A,B,C are collinear.<br>
If ${\rm orientation}(A, B, C) > 0$, the points A,B,C are in counterclockwise order.<br>
If ${\rm orientation}(A, B, C) < 0$, the points A,B,C are in clockwise order. 

The orientation function can be used to check if two line segments, P and Q, intersect.

#### Test for Intersecting Lines

<img src="https://github.com/engmaths/SEMT10002_2025/blob/main/media/week_10/intersecting_lines.png?raw=true" width="40%">

Line segments P and Q **intersect** if:

${\rm sign}({\rm orientation}(P_1, P_2, Q_1)) \neq {\rm sign}({\rm orientation}(P_1, P_2, Q_2))$

AND

${\rm sign}({\rm orientation}(Q_1, Q_2, P_1)) \neq {\rm sign}({\rm orientation}(Q_1, Q_2, P_2))$

where ${\rm sign}$ outputs negative, positive or zero i.e. tells us if ${\rm orientation}(A, B, C)$ is $<0$, $>0$ or $0$

#### Special case: Test for Collinear overlap

Line segments P and Q are **collinear** if:

 ${\rm orientation}(Q_1, Q_2, P_1) = {\rm orientation}(Q_1, Q_2, P_2)={\rm orientation}(P_1, P_2, Q_1) = {\rm orientation}(P_1, P_2, Q_2) = 0$

<img src="https://github.com/engmaths/SEMT10002_2025/blob/main/media/week_10/colinear_lines.png?raw=true" width="30%">

 To overlap (intersect) they must *also* satisfy the condition: 

 ${\rm min}(x_{P1}, x_{P2}) \leq x_{Q1} \leq {\rm max}(x_{P1}, x_{P2})$<br>
 ${\rm min}(y_{P1}, y_{P2}) \leq y_{Q1} \leq {\rm max}(y_{P1}, y_{P2})$

 OR

${\rm min}(x_{P1}, x_{P2}) \leq x_{Q2} \leq {\rm max}(x_{P1}, x_{P2})$<br>
${\rm min}(y_{P1}, y_{P2}) \leq y_{Q2} \leq {\rm max}(y_{P1}, y_{P2})$

***

Use the orientation function, and tests for intersecting lines and collinear overlap to modify the `detect_obstacles` function in the robot simulation

**Tips for getting started:**

>Download *ExerciseP4.py*, which contains the sample answers for P1, P2 and P3. 

>First writing code to check if two lines intersect as a seperate program, and test that this works, before integrating this with the robot simulation. 

>Try modelling just one sensor to begin with, pointing at $0$ radians *relative to* the robot heading. Can you determine the start and end points of this line?

>Our simulation includes 3 obstacles (with 4 lines representing the edges of the obstacle), and 4 lines representing the egdes of the map. This means you'll need to check the **sensor line** against a total of 16 lines. Can you determine the start and end points of these lines? We recommend you start by focusing on a single line and then extend your code to check all lines.

>The latest version of robot_plotter.py contains an updated version of the `show_plot` function that takes an *optional* argument `sensors`.<br> The format of sensors should be a list of tuples representing the start and end coordinates of each sensor vector. It can be useful to visualise the sensors to check your program is behaving as expected

```python
sensors = [((sensor1_start_x, sensor1_start_y),(sensor1_end_x, sensor1_end_y)), 
           ((sensor2_start_x, sensor2_start_y),(sensor2_end_x, sensor2_end_y)), 
           ((sensor3_start_x, sensor3_start_y),(sensor3_end_x, sensor3_end_y))
           ] 

show_plot(map_coords, goal=goal, obstacles=obstacles, pause=0.1, sensors=sensors)
```
<img src="https://github.com/engmaths/SEMT10002_2025/blob/main/media/week_10/show_plot_new_functionality.png?raw=true" width="30%">

***

## Testing your program 


We will use the following test cases to check that your code works

You can run these tests on your pogram by running the program *test_ExerciseP4.py* in the same directory as your program

**Map boundaries:** x: 0–5000, y: 0–5000  

**Obstacles:**  
- Obstacle 1: ((100, 2250), (4000, 500))  
- Obstacle 2: ((3000, 3000), (800, 1500))  
- Obstacle 3: ((1500, 500), (600, 600)) 


| Test | robot_x | robot_y | sensor_angles | sensor_range | Expected Output | Description |
|---------|---------|---------|---------------|--------------|----------------|-------|
|Test 1| 2000 | 1000 | [0] | 300 | (True, "obstacle") | Sensor pointing vertical up towards **obstacle** 1 |
|Test 2| 1400 | 800 | [π/2] | 300 | (True, "obstacle") | Sensor pointing horizontal right towards **obstacle** 3 |
|Test 3| 10 | 2000 | [-π/2] | 300 | (True, "wall") | Sensor pointing horizontal left toward left **wall** |
|Test 4| 3000 | 30 | [π] | 300 | (True, "wall") | Sensor pointing  vertical down toward bottom **wall** |
|Test 5| 1400 | 800 | [-π/2] | 300 | (False, None) | Robot near **obstacle** 3 but sensor pointing away |
|Test 6| 1400 | 1100 | [0, π/4, π/2] | 300 | (True, "obstacle") | Only right sensor hits **obstacle** 3 |
|Test 7| 3000 | 300 | [-π/4, 0, π/4] | 300 | (False, None) | Robot in open area, **no collisions** |
|Test 8| 4000 | 1000 | [-π/4, 0, π/4] | 300 | (False, None) | Robot in open area, **no collisions** |



## Assessment

We will assess your code using the automatic marking script test_code_portfolio.py. This script will run your code and test it against each of the 8 test cases listed above. Your mark will be based on the number of tests your code can successfully pass. There are no marks for readability in this assessment. The mark scheme for P4 is as follows:

| Number of tests passed | Mark |
| ---------------------- | ---- |
| 2 | 25% |
| 4 | 50% |
| 6 | 75% |
| 8 | 100% |
